# Introduction

This notebook demonstrates the Wise Platform **Quote - Recipient - Transfer - Found** flow via Wise Platofrm API. 

To begin obtain your API key for the Wise Sandbox environment at [https://sandbox.transferwise.tech/home](https://sandbox.transferwise.tech/home). 
Navigate to **Your account** -> **Integrations and tools** -> **API tokens** to generate or retrieve your API key.

Ensure that the API key is stored securely and is not exposed in public repositories or shared with unauthorized individuals.

In [1]:
# Wise API Key
api_key = "your_wise_api_key"

# Wise Client

Wise Client is a utility designed to streamline the process of sending and receiving API messages to and from the Wise Platform API. 

In [13]:
import pandas as pd
pd.set_option('display.max_rows', None)

import requests

# Wise API Client
class WiseAPI:
  BASE_URL = "https://api.sandbox.transferwise.tech"
  def __init__(self, api_key):
    self.api_key = api_key
    self.headers = {"Authorization": f"Bearer {self.api_key}"}

  def get(self, endpoint):
    url = f"{self.BASE_URL}/{endpoint}"
    response = requests.get(url, headers=self.headers)
    response.raise_for_status()
    return response.json()

  def post(self, endpoint, payload):
    url = f"{self.BASE_URL}/{endpoint}"
    response = requests.post(url, json=payload, headers=self.headers)
    response.raise_for_status()
    return response.json()

wise= WiseAPI(api_key)

# Step 1: Get user profile

Profiles are connected to a User account and are either personal or business.
> https://docs.wise.com/api-docs/api-reference/profile

In [ ]:
profiles = wise.get(endpoint = "/v2/profiles")
profiles_df = pd.json_normalize(profiles)
profiles_df.T

# Step 2: Create a quote

The quote resource defines the basic information required for a Wise transfer - the currencies to send between, the amount to send and the profile who is sending the money. The profile must be included when creating a quote.

> https://docs.wise.com/api-docs/api-reference/quote

In [ ]:
profile_id = profiles_df['id'][0]
quote = wise.post(endpoint = f"/v3/profiles/{profile_id}/quotes", payload = {
      "sourceCurrency": "USD",
      "targetCurrency": "GBP",
      "sourceAmount": 100,
      "targetAmount": None,
      "targetAccount": None,
      "preferredPayIn": "BALANCE"
    })
quote_df = pd.json_normalize(quote)
quote_df.T

### List Payment Options

In [ ]:
df = pd.json_normalize(quote_df['paymentOptions'][0])
df

# Step 3: Create a Recipient Account

Recipient or beneficiary is the one who will receive the funds.

> https://docs.wise.com/api-docs/api-reference/recipient

In [ ]:
recipient = wise.post(endpoint = f"/v1/accounts", payload = {
          "currency": "GBP",
          "type": "sort_code",
          "profile": str(profile_id),
          "ownedByCustomer": True,
          "accountHolderName": "John Doe",
           "details": {
              "legalType": "PRIVATE",
              "sortCode": "040075",
              "accountNumber": "37778842",
              "dateOfBirth": "1961-01-01"
           }
         })

recipient_df = pd.json_normalize(recipient)
recipient_df.T

# Step 4: Create a transfer

A transfer is a payment order to recipient account based on a quote.
> https://docs.wise.com/api-docs/api-reference/transfer

In [ ]:
import uuid

transfer = wise.post(endpoint = f"/v1/transfers", payload = {
      "targetAccount": recipient['id'],
      "quoteUuid": quote['id'],
      "customerTransactionId": str(uuid.uuid4()),
    })

transfer_df = pd.json_normalize(transfer)
transfer_df.T

# Step 5: Fund the transfer

This is the final step for executing payouts when using a balance with Wise. Upon calling the endpoint, Wise will begin the processing of the transfer, depending on the status of funds.

> https://docs.wise.com/api-docs/api-reference/transfer#fund

In [ ]:
funding = wise.post(endpoint = f"/v3/profiles/{profile_id}/transfers/{transfer['id']}/payments", payload = {   
    "type":"BALANCE"
  })
funding_df = pd.json_normalize(funding)
funding_df.T
